In [1]:
import pandas as pd
import requests
from config import DATA_PATH, API_URL, COORDINATES

In [2]:
def weather_request(year: str):
    params = {
        "latitude": COORDINATES["latitude"],
        "longitude": COORDINATES["longitude"],
        "start_date": f"{year}-01-01",
        "end_date": f"{year}-12-31",
        "hourly": "temperature_2m,windspeed_10m,precipitation",
        "timezone": "Europe/Moscow"
    }

    return requests.get(API_URL, params).json()
    

In [5]:
daily_dfs = dict.fromkeys(["2023", "2024"])
for key in daily_dfs:
    data = weather_request(key)
    df = pd.DataFrame({
        "day": pd.to_datetime(data["hourly"]["time"]).day_of_year,
        "temperature": data["hourly"]["temperature_2m"],
        "wind_speed": data["hourly"]["windspeed_10m"],
        "precipitation": data["hourly"]["precipitation"]
    })

    daily_df = df.groupby(by="day").mean().reset_index()
    new_min, new_max = 1, 100
    old_min, old_max = daily_df['precipitation'].min(), daily_df['precipitation'].max()
    daily_df['precipitation'] = (daily_df['precipitation'] - old_min) / (old_max - old_min) * (new_max - new_min) + new_min


In [6]:
for key, val in daily_dfs.items():
    val.to_csv(f"{DATA_PATH['parsed']}weather_{key}.csv", sep=",", index=False)

AttributeError: 'NoneType' object has no attribute 'to_csv'